In [1]:
import torch
from typing import List, Union, Tuple
from functools import partial
import json
from transformer_lens.HookedTransformer import HookedTransformer

from algebraic_value_editing.completion_utils import print_n_comparisons
from algebraic_value_editing.prompt_utils import get_x_vector

In [2]:
model_name = "gpt2-small"
# model_name = "gpt-j-6B"
# model_name = "pythia-2.8b-deduped"

# GPT-J-6B can't load onto GPU RAM of Colab
device: str = "cuda" if (torch.cuda.is_available() and model_name != "gpt-j-6B") else "cpu"
model: HookedTransformer = HookedTransformer.from_pretrained(model_name, device="cpu").to(device)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cpu


## Load Dataset

In [3]:
stories = {}

In [4]:
# Specify the file path
file_path = './datasets/fantasy_200.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
  # Load the JSON data from the file
  dataset_fantasy = json.load(file)

  stories["fantasy"] = dataset_fantasy

## Run Model (!)!!

In [45]:
from algebraic_value_editing.dataset_utils import ActivationAdditionDataset

activation_addition_dataset = [ActivationAdditionDataset(
    coeff=0.9,
    act_name=0,
    prompt=stories["fantasy"],
    from_dataset=True,
)]

In [46]:
activation_addition_dataset[0].location

'blocks.0.hook_resid_pre'

In [47]:
default_kwargs = {'temperature': 1, 'freq_penalty': 1, 'top_p': .3, 'model': model}
get_x_vector_preset = partial(get_x_vector, pad_method="tokens_left",
                              model=model,
                              custom_pad_id=model.to_single_token(" "))

In [48]:
print_n_comparisons(prompt="Yesterday, my son brought home a pet goose. This was the start of",
                    tokens_to_generate=80, activation_additions=activation_addition_dataset,
                    num_comparisons=8, seed=0, **default_kwargs)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                    Unsteered completions                     |                     Steered completions                      |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Yesterday, my son brought home a pet goose. This was the   |   Yesterday, my son brought home a pet goose. This was the   |
|   start of his life as a dog. He was raised by his mother    |                      start of the day.                       |
| and father in the U.S., and I had to take him out for walks  |                                                              |
| with me every day because he was so small. He has never been |   I got to see it in action and I'm so happy to be able to   |
| shy about taking care of himself, even when he is scared or  | share it with you. It's a big goose tha